### example

asyncio leverages 4 components to pass controls
- coroutine.send(arg) 
    - method to start or resume a coroutine
    - arg
        - if coro is used for first time, arg must be None
        - if coro was paused and is now being resumed, arg will be the return value of the yield that originally paused execution
- In a coroutine, the only way to cede control is to await an awaitable that yields in its `__await__` method.
    - yield pauses execution and returns control to caller
- when coroutine finishes, it raises StopIteration with return value

In [ ]:
class Rock:
    def __await__(self):
        print(f"Rock.__await__ called.")

        value_sent_in = yield 7 
        # A3.this yield pauses coroutine execution; 
        # B2. resume from where yielded, which returns the arg from send()

        print(f"Rock.__await__ resuming with value: {value_sent_in}.")
        return value_sent_in

async def main():
    print("Beginning coro.")
    rock = Rock()
    print("Awaiting the awaitable rock...")

    value_from_rock = await rock 
    # A2. calls __await__; 
    # A4. propagates yielded value

    print(f"coro received value: {value_from_rock} from rock.")
    return value_from_rock
    # B3. when coro finishes, it raises StopIteration with return value

coroutine = main()
print(f"calling send with: None.")

intermediate_result = coroutine.send(None) 
# A1. start coro with send(None)
# A5. receives yielded value

print(f"{intermediate_result = }\n")

try:
    print(f"calling send with: 42.")
    coroutine.send(42) # B1. Resume coro, send 42
except StopIteration as e:
    returned_value = e.value
    print(f"{returned_value = }")

calling send with: None.
Beginning coro.
Awaiting the awaitable rock...
Rock.__await__ called.
intermediate_result = 7

calling send with: 42.
Rock.__await__ resuming with value: 42.
coro received value: 42 from rock.
returned_value = 42


### generator

coro implemented via generator

- generator uses yield to pause execution (A), output (B) and receive values (C)

In [59]:
def gen():
    received = yield 1 # A
    print(f"gen received: {received}")
    received = yield 2
    print(f"gen received again: {received}")

In [60]:
g = gen()
g

<generator object gen at 0x00000226E4CC4E10>

In [61]:
[i for i in dir(g) if not i.startswith('_')]

['close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_suspended',
 'gi_yieldfrom',
 'send',
 'throw']

In [62]:
print(f"{next(g) = }") # B
g.gi_frame.f_locals

next(g) = 1


{}

In [63]:
print(f"{g.send(9) = }") # C
g.gi_frame.f_locals

gen received: 9
g.send(9) = 2


{'received': 9}

In [64]:
try:
    g.send(99)
except StopIteration:
    print("Generator has completed.")

gen received again: 99
Generator has completed.


### tie back to event loop (untested)

1. **Event Loop calls `coroutine.send(None)`**
   - Just like your Rock example, but event loop is the caller
   - Starts or resumes coroutine execution

2. **Coroutine hits `await` statement**
   - Calls `__await__()` on the awaitable object
   - Awaitable yields control (like your Rock yielding 7)

3. **Event Loop receives yielded value**
   - Could be a socket, file descriptor, timer, etc.
   - Event loop knows what I/O operation to wait for

4. **Event Loop manages I/O**
   - Uses OS primitives (epoll, select, IOCP) to wait efficiently
   - Doesn't block - can switch to other ready coroutines

5. **I/O completes**
   - Event loop calls `coroutine.send(result)` to resume
   - Coroutine continues from where it yielded


In [ ]:
# # Event Loop acts as the orchestrator that calls send() on coroutines
# import asyncio
# import time

# class IOOperation:
#     """Simulates an I/O operation that the event loop can manage"""
#     def __init__(self, operation_name, duration):
#         self.operation_name = operation_name
#         self.duration = duration
        
#     def __await__(self):
#         print(f"Starting {self.operation_name}...")
        
#         # This yield tells event loop: "I'm waiting for I/O, come back later"
#         yield self  # Event loop receives this object
        
#         # When resumed, simulate I/O completion
#         print(f"{self.operation_name} completed!")
#         return f"Result from {self.operation_name}"

# async def fetch_data(url):
#     """Coroutine that performs I/O operations"""
#     print(f"Fetching {url}")
    
#     # await triggers __await__, which yields control to event loop
#     result = await IOOperation(f"HTTP request to {url}", 2)
    
#     print(f"Got: {result}")
#     return result

# async def main():
#     """Main coroutine that coordinates multiple I/O operations"""
#     # Create multiple coroutines
#     tasks = [
#         fetch_data("api.example.com/users"),
#         fetch_data("api.example.com/posts"),
#         fetch_data("api.example.com/comments")
#     ]
    
#     # Event loop will manage these concurrently
#     results = await asyncio.gather(*tasks)
#     return results

# # Simplified event loop simulation
# class SimpleEventLoop:
#     def __init__(self):
#         self.ready_queue = []
#         self.waiting_ios = []
    
#     def run_coroutine(self, coro):
#         """Main event loop logic"""
#         self.ready_queue.append(coro)
        
#         while self.ready_queue or self.waiting_ios:
#             # Process ready coroutines
#             while self.ready_queue:
#                 current_coro = self.ready_queue.pop(0)
                
#                 try:
#                     # This is like coroutine.send(None) or coroutine.send(result)
#                     yielded_value = current_coro.send(None)
                    
#                     # If coroutine yielded an I/O operation
#                     if isinstance(yielded_value, IOOperation):
#                         print(f"Event loop: Got I/O operation {yielded_value.operation_name}")
#                         # Simulate I/O completion after some time
#                         self.waiting_ios.append((current_coro, yielded_value))
                    
#                 except StopIteration as e:
#                     print(f"Coroutine finished with result: {e.value}")
#                     return e.value
            
#             # Simulate I/O completion (in real event loop, this is epoll/select/IOCP)
#             if self.waiting_ios:
#                 completed_coro, io_op = self.waiting_ios.pop(0)
#                 print(f"Event loop: I/O {io_op.operation_name} completed, resuming coroutine")
#                 self.ready_queue.append(completed_coro)

In [ ]:
# # Example showing the connection:
# print("=== Manual Event Loop Simulation ===")
# loop = SimpleEventLoop()
# # This demonstrates how event loop uses send() to manage coroutines
# loop.run_coroutine(main())

=== Manual Event Loop Simulation ===


Fetching api.example.com/users
Starting HTTP request to api.example.com/users...
Fetching api.example.com/posts
Starting HTTP request to api.example.com/posts...
Fetching api.example.com/comments
Starting HTTP request to api.example.com/comments...
